## Exploring probability

In this notebook, we explore some ideas of probability that will help us make inference in the future. This workbook starts with an exercise building a data generating process that flips coins, recording "H" or "T". We then practice building statistics based on the flips. In this context, the word "statistic" just means some sort of numerical summary of a data set. Statistics can be really simple, like the number of rows in a table, or really complicated, like an accuracy measure of a fancy machine learning model. 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sequences import sequence_1, sequence_2

%matplotlib inline


In [ ]:
def do_flips(n=100,seed=None) : 
    """
        Inputs: 
            * n is the number of simulations you want. 
            * seed is an integer that ensures reproducible input
    
        Outputs: 
            * a list of length `n` with randomly chosen Heads and Tails, marked with 
            "H" and "T".

    """

    if seed : 
        np.random.seed(seed)

    return np.random.choice(["H","T"],size=n,replace=True).tolist()


def print_flips(flips,col_width=10) : 
    """
        Given a list of flips, this function prints 
        them in columns col_width wide going down the page.
    """
    sublists = [flips[i:i+col_width] for i in range(0, len(flips), col_width)]

    output = "\n".join([" ".join(row) for row in sublists])
    
    print(output)

    # This functions return is a *side effect*. The printing of the flips.
    return None


## Exploring coin flips

Write a function that takes as input a list of "H" and "T", produced by your `do_flips` function and outputs the *number* of flips that came up heads. Run this 10-20 times flipping 10 coins. Then running 10-20 times flipping 100 coins. 

#### Question: 

* What seems more unusual? 3 heads out of 10 or 30 heads out of 100?  

In [ ]:
def count_heads(flips) :
    """
        Returns the number of heads in a list or string.
    """

    # Your code here. Return the number of heads

    return 0 
    
    return head_count


## Assertion statements are a great way to test your code. 
assert(count_heads("HHHHHTTTTT")==5)
assert(count_heads("H")==1)
assert(count_heads("T")==0)
assert(count_heads("HTHTHTHTHTHTHT")==7)


In [ ]:
count_heads(do_flips(10))

In [ ]:
count_heads(do_flips(100))


Q: What seems more unusual? 3 heads out of 10 or 30 heads out of 100?  
A: <!-- Put your answer around here. What's more rare? 3 out 10 or 30 out of 100? -->

---

### A new statistic

In the space below, try to write a statistic that counts the number of times that two heads appear in sequence. So "HHHTHH" would have three squences in spots 0-1, 1-2, and 4-5.

In [ ]:
def count_two_heads(flips) : 

    # Your code here

    return 0

assert(count_two_heads("HHHTHH")==3)
assert(count_two_heads("H")==0)
assert(count_two_heads("TTHHTTHHTTHH")==3)
assert(count_two_heads("HTHTHTHTHTHTHT")==0)
assert(count_two_heads("HHHHHTTTTTT")==4)


--- 

## Some Exploration

Call `count_heads` and `count_two_heads` on `sequence_1` and `sequence_2`, and some instances of `do_flips(100). Does anything stand out at this point?

In [ ]:
print(count_two_heads(sequence_1))
print(count_two_heads(sequence_2))

In [ ]:
flips = do_flips(100)
print(count_heads(flips))
print(count_two_heads(flips))

### Question: 

Does anything about our squences stand out in either of these statistics for you? 

---

Here's a spot where you can pause if we're in class. We'll sync up and talk for a bit.


---

### Simulations at scale

The beauty of exploring probability with computers is that we can do _lots_ of simulations to understand what's going on. Let's do 10,000 sequences of 10 and 100 coin flips respectively, so that we can look at the results. 

Side note: we're doing 10000 * 10 + 10000 * 100 coin flips, which is 1.1M coin flips. How long would that take you? 

In [ ]:
heads_10 = []
heads_100 = []

for _ in range(10000) : 
    heads_10.append(count_heads(do_flips(10)))
    heads_100.append(count_heads(do_flips(100)))

In [ ]:
# This is a pretty common way for me to make data frames. Use base Python
# to make lists or dictionaries, then put the results in a DF to get
# summary stats and plotting.

experiment_results = pd.DataFrame({"heads_10":heads_10,
                                   "heads_100":heads_100})

In [ ]:
experiment_results['heads_10'].value_counts().sort_index()

In [ ]:
experiment_results['heads_100'].value_counts().sort_index()
# Printing is not the way to go here

In [ ]:
num_flips = 100

plt.bar(x = experiment_results['heads_100'].value_counts().index,
        height=experiment_results['heads_100'].value_counts())
plt.xlabel("Number of Heads")
plt.ylabel("Count")
plt.title(f"Number of heads in {num_flips}")

plt.show()

In [ ]:
num_flips = 10

plt.bar(x = experiment_results['heads_10'].value_counts().index,
        height=experiment_results['heads_10'].value_counts())
plt.xlabel("Number of Heads")
plt.ylabel("Count")
plt.title(f"Number of heads in {num_flips}")

plt.show()

It's going to be handy to have a function that does this plotting, so I'm going to write one.

In [ ]:
# A function to plot our simulations 

def plot_simulations(statistic_values, 
                     statistic_labels,
                     chart_title = None,  
                     test_value=None) : 
    """
        This function takes a statistic and it's x-axis label and builds 
        a bar chart of all the values. If you include a `test_value`, then 
        that value will get a vertical line on the chart and you'll learn
        what fraction of values are as extreme or more extreme than that value.
    
        You can also pass in a chart title.
    """

    plt.bar(x=statistic_labels, 
            height=statistic_values, 
            color="skyblue", 
            edgecolor="black")
    plt.xlabel("Statistical Value")
    plt.ylabel("Count")


    extremity = None
    if test_value is not None:
        mean_val = np.average(statistic_labels, weights=statistic_values)

        if test_value <= mean_val:
            extremity = np.sum(statistic_values[np.array(statistic_labels) <= test_value]) / np.sum(statistic_values)
            arrow_dir = -1  # arrow points left
        else:
            extremity = np.sum(statistic_values[np.array(statistic_labels) >= test_value]) / np.sum(statistic_values)
            arrow_dir = 1   # arrow points right

        ax = plt.gca()
        ymax = ax.get_ylim()[1]
        xmin, xmax = ax.get_xlim()
        offset = 0.2 * (xmax - xmin)  

        plt.axvline(x=test_value, color="red", linestyle="--",
                    label=f"{extremity:.2%} as extreme")


        # Add arrow annotation
        plt.annotate(
            "extreme tail",
            xy=(test_value, ymax*0.8),                         
            xytext=(test_value + arrow_dir*offset, ymax*0.8),   
            arrowprops=dict(facecolor="red", arrowstyle="<-"),
            color="red", ha="center"
        )

    if chart_title:
        plt.title(chart_title)

    if test_value is not None:
        plt.legend()

    plt.show()
    
    return extremity



In [ ]:
plot_simulations(experiment_results['heads_10'].value_counts(),experiment_results['heads_10'].value_counts().index)

In [ ]:
plot_simulations(experiment_results['heads_100'].value_counts(),experiment_results['heads_100'].value_counts().index)

Q: Now that you've seen 10,000 coin flip sequences for both scenarios, which seems more common?
A: <!-- Your answer around here. Remember, remove these comment characters. -->

---

# A Brief Diversion

It will be important for you to understand some of the power of the `random.random` function, which returns pseudo-random numbers from between 0 and 1. Use the cell below to generate some examples from this function.


In [ ]:
np.random.random()

Now use the cell below to write a function that allows you to simulate a coin with a probability of heads of something other than 50%. The easiest way is to generate a random value and compare it to your desired heads proportion. If the random value is smaller than the proportion, then mark the flip heads.

In [ ]:
def do_flips_2(size,heads_probability=0.5,seed=None) : 

    # I've got it started for
    if seed : 
        np.random.seed(seed)

    holder = [None] * size


    for idx in range(size) : 
        # here's where you will fill in `holder` with H and T
        pass

    return holder

In [ ]:
unfair_flips = []

for _ in range(1000) : 
    unfair_flips.append(count_heads(do_flips_2(100,heads_probability=1/30)))

experiment_results = pd.DataFrame({'heads':unfair_flips})



In [ ]:
plot_simulations(experiment_results.value_counts("heads"),experiment_results.value_counts("heads").index)

--- 

### Looking at Rare Events

Let's return to our question: 

* What seems more unusual? 3 heads out of 10 or 30 heads out of 100?  

Let's regenerate the data and plot these, looking at our test values.

In [ ]:
heads_10 = []
heads_100 = []

for _ in range(10000) : 
    heads_10.append(count_heads(do_flips(10)))
    heads_100.append(count_heads(do_flips(100)))

experiment_results = pd.DataFrame({'heads_10':heads_10,
                                  'heads_100':heads_100})

In [ ]:
plot_simulations(experiment_results['heads_10'].value_counts(),
                 experiment_results['heads_10'].value_counts().index,
                 chart_title = "Number of Heads in 10 Flips",
                 test_value=3)

In [ ]:
plot_simulations(experiment_results['heads_100'].value_counts(),
                 experiment_results['heads_100'].value_counts().index,
                 chart_title = "Number of Heads in 100 Flips",
                 test_value = 39)

### Answer

<!-- Your answer here --> 

## Two sequences

You've been given two sequences. One of them isn't really random. Let's see if we can differentiate them based on number of heads.

In [ ]:
print(f"Sequence 1 has {count_heads(sequence_1)} heads.")
print(f"Sequence 2 has {count_heads(sequence_2)} heads.")

Doesn't look very likely that we'll differentiate them based on heads! But let's run them through.

In [ ]:
plot_simulations(experiment_results["heads_100"].value_counts(),
                 experiment_results["heads_100"].value_counts().index,
                 test_value=count_heads(sequence_1),
                 chart_title="Testing Sequence 1")

In [ ]:
plot_simulations(experiment_results["heads_100"].value_counts(),
                 experiment_results["heads_100"].value_counts().index,
                 test_value=count_heads(sequence_2),
                 chart_title="Testing Sequence 2")

## Observations

Sequence 2 _is_ a bit more extreme, but it's still not very extreme.

## Custom Statistics

Think about some statistics that might differentiate the fake data from the real. Build functions for them and see if you can find something that's significant.

In [ ]:
# Write one or two functions here

In [ ]:
# Repeat our work to fill up lists and store them in a DataFrame

In [ ]:
# Then plot them and compare to the values from our sequences

In [ ]:
experiment_results['runs'].value_counts().sort_index()

## Results

<!-- Your thoughts here --> 
